### Vorwort:
---
Das nachfolgende Notebook richtet sich an interessierte Schülerinnen und Schüler der Oberstufe, welche Interesse an Teilchenphysik haben und einen kleinen Einblick in eine Analyse suchen.

Es bietet einen kleinen Ausschnitt aus einer möglichen sehr vereinfachten Suche nach dem Higgs Boson mithilfe der öffentlich bereitgestellten Datensätze.

Die Bearbeitung des Notebooks sollte mit einer fachlichen Betreuung erfolgen, um entstehende Fragen beantworten zu können. Für das Vorwissen sollte zumindest die CMS Masterclass besucht worden sein. Ebenfalls richtet sich dieses Notebook an Schülerinnen und Schüler, die mit der Programmiersprache Python vertraut sind und die Grundlagen beherrschen. Sollte dies nicht der Fall sein, so kann auf die entsprechenden Notebooks zu Python in diesem Repository oder anderweitig zugängliche Tutorien verwiesen werden.

Die Gliederung des Notebooks ist in mehrere Passagen unterteilt. Diese wechseln sich zwischen Abschnitten zum selbstständigem Arbeiten und ausprobieren und Abschnitten bei denen entweder ausführliche Erklärungen nach Interesse vorgestellt oder aufkommende Fragen beantwortet werden können.


#### Work in Progress:

Dies ist (noch) nicht die finale Version des Notebooks und der Aufgabenstellung. Die finale Variante wird nach der Fertigstellung auf dem Main-Branch zu finden sein. 

Die Kommentare innerhalb der Code-Passagen sind auf Englisch verfasst. 

## Suche nach dem Higgs Boson
### Ein vereinfachter Ausschnitt aus der Suche im $\mathrm{H} \rightarrow ZZ \rightarrow 4\ell$ Kanal


Das Higgs Boson kann in viele unterschiedliche Teilchen zerfallen, da dieser an die Masse koppelt. Ein möglicher Zerfall wäre zum Beispiel durch zwei Z-Bosonen möglich, welche dann anschließend selbst weiter zerfallen. Die Zerfallsprodukte dieser Z-Bosonen - in unserem Fall werden es Elektronen und Myonen sein, können im Detektor nachgewiesen werden.

Das Ziel wird es sein, ausgehend von diesen Elektronen und Myonen ein Algorithmus zu bauen, welcher diese Z-Bosonen rekonstruiert. Mithilfe dieser zwei Kandidaten der Z-Bosonen können Kandidaten für ein Higgs Boson rekonstruiert werden, welcher anschließend in Form von einer Überhöhung in einer flachen Verteilung gefunden werden kann.

Ob ein solcher Überschuss beobachtet werden kann und wie signifikant dieser ist, wird sich am Ende des Notebooks ergeben.

In [1]:
from IPython.display import display
import numpy as np
import pandas as pd

from utils import FourVecAccessor, EventFilter, LeptonFilter, get_leptons_by_flavour


Als ersten Schritt laden wir zwei Datensätze, die unsere Erwartung an zwei Prozesse beschreiben. In der Teilchenphysik werden diese Erwartungen in Form von Simulationen beschrieben, welche aus der Theorie gezogen werden können.

In diesem Notebook ist es einmal die Simulation von einem erwarteten Higgs Boson, mit einer Masse von $125\,\mathrm{GeV}$, und anderen Untergrundprozessen, welche in ihrem Endzustand nicht von einem Higgs Boson unterschieden werden können.

Auch wird neben diesen beiden Simulationen noch ein Datensatz mit einer durchgeführten Messung geladen, da die nachfolgenden Schritte der Rekonstruktion der Z-Bosonen und der Higgs Kandidaten auch für diese durchgeführt werden sollten. Ein genauerer Blick auf diesen Daten erfolgt erst zu einem späteren Zeitpunkt.

In [2]:
dfs = pd.read_csv("data/MC_2012_H_to_ZZ_to_4L_[100,151].csv.gz", header=[0, 1])  # signal MC
dfb = pd.read_csv("data/MC_2012_ZZ_to_4L_[100,151].csv.gz", header=[0, 1])  # signal MC
dfm = pd.read_csv("data/CMS_Run2012[B,C]_[100,151].csv.gz", header=[0, 1])

print("Signal simulation")
display(dfs.head())

print("Background simulation")
display(dfb.head())

Signal simulation


event_information lepton_0                                           \
            channel        E       px      py       pz flavour charge   
0                 2   40.520 -36.9470  14.478   8.1968     1.0    1.0   
1                 0   45.575 -33.0500 -30.283   8.2250     0.0   -1.0   
2                 0   63.087 -23.3130  23.232  53.8220     0.0   -1.0   
3                 0   43.890   9.2923 -40.068  15.3150     0.0   -1.0   
4                 0   84.498 -28.3290 -17.593 -77.6390     0.0    1.0   

            lepton_1           ...  Z1      Z2             four_lep          \
   relpfiso        E       px  ...  py  pz   E  px  py  pz        E  px  py   
0  0.107879   32.044 -10.1480  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
1  0.008728   57.617   9.3309  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
2  0.083510   35.279  26.6930  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
3  0.019456   30.698 -21.0470  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
4  0.070871   33.163  30.6340  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   

       
   pz  
0 NaN  
1 NaN  
2 NaN  
3 NaN  
4 NaN  

[5 rows x 41 columns]

Background simulation


event_information lepton_0                                            \
            channel        E      px       py        pz flavour charge   
0                 0  131.840  49.035 -24.0350 -120.0000     0.0   -1.0   
1                 1  112.090  15.444 -28.5690 -107.2800     1.0   -1.0   
2                 0   51.086  12.700  40.9880  -27.7210     0.0   -1.0   
3                 0   41.822  41.330   6.2322   -1.4221     0.0   -1.0   
4                 2   56.155  34.641 -21.4060  -38.6670     1.0    1.0   

            lepton_1           ...  Z1      Z2             four_lep          \
   relpfiso        E       px  ...  py  pz   E  px  py  pz        E  px  py   
0  0.113952   23.322  13.3900  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
1  0.022230   25.031   4.5001  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
2  0.063375  111.560 -19.1620  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
3  0.015386   89.043 -27.6750  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   
4  0.102327  165.820 -28.0560  ... NaN NaN NaN NaN NaN NaN      NaN NaN NaN   

       
   pz  
0 NaN  
1 NaN  
2 NaN  
3 NaN  
4 NaN  

[5 rows x 41 columns]

Diese Datensätze beinhalten immer vier Leptonen
`lepton_0`, `lepton_1`, `lepton_2`, `lepton_3` und `lepton_4` auf die zum Beispiel mit `dataframe["lepton_0"]` oder `dataframe.lepton_0` problemlos zugegriffen werden kann.

In [ ]:
dfs["lepton_0"]

Auf einzelnen interessante Größen wie die Energie und den Impuls kann analog wie auf einzelne Leptonen über `dataframe["lepton_0"]["px"]` zugegriffen werden. Neben der Energie und den Impulsen in einzelne Richtungen gibt es zudem bei jedem Leptonen einen `"flavour"` welcher zwischen Myonen `0` und Elektronen `1` unterscheidet, sowie der elektrischen Ladung `charge`. Für die nachfolgende Rekonstruktion werden zunächst nur diese Größen benötigt. Die Größe `relpfiso` wird erst bei der Anwendung eines Filters benötigt und wird bei der Anwendung nach Interesse durch einen weiterführenden Vortrag besprochen.

Um damit etwas vertrauter zu werden: Zeige die Impulskomponente in z-Richtung von dritten Lepton an.

In [ ]:
# Your Code goes here


Neben den Leptonen beinhalten die Datensätze noch die Informationen für die zu rekonstruierenden Objekte `"Z1"` und `"Z2"` und den daraus resultierenden Objekt, welcher aus vier Leptonen rekonstruiert wurde: `"four_lep"`, also einem Higgs Kandidaten. Diese Größen werden in der anschließenden Rekonstruktion für jedes Ereignis berechnet und eingetragen.




Noch zu erwähnen wäre die Unterscheidung nach den unterschiedlichen Zerfallskanälen mit `dataframe["event_informations"]["channel"]`. Es wird zwischen den Zerfällen in vier Leptonen mit gleichen Flavour (vier Myonen: `0`; vier Elektronen: `1`) und einem Mischkanal (zwei Myonen, zwei Elektronen) `2` unterschieden.



Bevor der Rekonstruktionsschitt angegangen wird soll zunächst einmal eine einfache Implimentierung zur Berechnung der invarianten Masse in Form einer Python Funktion durchgeführt werden. 

* Was ist die invariante Masse?

* Wie kann die invariante Masse von zwei Zerfallsprodukten berechnet werden? 

* Wie kann ich Formel auch für viele einzelne Zerfallsprodukte motivieren?

In [ ]:
# complete this function
def calculate_invariant_mass(energy, px, py, pz):
    # calculate the invariant mass given the energy, px, py, pz of a particle
    return 0.0

# possible Variables for testing... what particle could it possible be?
particle_E = 97.5   # GeV
particle_px = 7.8   # GeV
particle_py = 32.5  # GeV
particle_pz = 43.9  # GeV
calculate_invariant_mass(particle_E, particle_px, particle_py, particle_pz)

Nun zu der eigentlichen Rekonstruktion:

Die Idee hinter der Vorgehensweise kann in Form von einem interaktiven Gespräch motiviert werden. Die Kodierung dieser Ideen erfolgt dann innerhalb der `perform_reconstruction` Funktion, in der einige Abschnitte im Code ergänzt werden sollen. Es können auch einzelne Code Abschnitte außerhalb der Funktion innerhalb neuer Zellen auf ihre Funktion isoliert ausprobiert werden.

In [ ]:
from tqdm import tqdm

# this is a rather bigger cell but bear with it, there are many similar lines that are introduced 
# to not confuse the flow. With this function, the reconstruction of both Z Bosons and the
# Higgs candidate is performed
def perform_reconstruction(df):
    z_boson_mass = 91.1876
    for i, event in tqdm(enumerate(df.iloc()), total=df.shape[0]):
        # If it is the Channel with four electrons or muons, we simply call muons
        # and electrons leptons and generalize for both decay channels
        if event.event_information.channel == 0 \
            or event.event_information.channel == 1:

            # How many possible combinations can be found when you pairwise combine
            # two leptons? Write them down in the same manner as the first two examples
            possible_pairs = [("12", event["lepton_0"], event["lepton_1"]),
                              ("13", event["lepton_0"], event["lepton_2"]),
                              # fill the remaining combinations the same way
                             ]
            # Consider the situation where the lepton charges are "+1", "-1", "-1", "+1",
            # how many possible combinations with an neutral electric charge can there be?

            # we will write those outcomes down and decide whether it is a Z1 candidate later
            Z_candidates = {"mass": [], "px": [], "py": [], "pz": [], "E": [], "label": []}

            for (label, lep1, lep2) in possible_pairs:
                # Access Energy, px, py, pz, charge via lep1.E, lep2.px and so on

                
                # proceed only if the combination of the two leptons has a neutral electric charge.
                # You can access the charge of a lepton via "lepton.charge"
                # why do you have to do this distinction?
                valid_electric_charge = False
                if valid_electric_charge:

                    Px = 0.0  # Calculate the total px considering both leptons
                    Py = 0.0  # Calculate the total py considering both leptons
                    Pz = 0.0  # Calculate the total pz considering both leptons
                    E = 0.0   # Calculate the total energy considering both leptons

                    # Use your implemented function to calculate the invariant mass
                    # of a Z candidate
                    M = calculate_invariant_mass(E, Px, Py, Pz)

                    # Collect the candidate
                    Z_candidates["mass"].append(M)
                    Z_candidates["px"].append(Px)
                    Z_candidates["py"].append(Py)
                    Z_candidates["pz"].append(Pz)
                    Z_candidates["E"].append(E)
                    Z_candidates["label"].append(label)


            # find the reconstructed Z boson that is nearest to the nominal Z Boson mass of
            # 91.1876 GeV. Provide the Index to determine the place of all corresponding
            # quantities of this reconstructed Z Boson.

            
            # we are interested in the distance of the given z_candidate mass and the nominal
            # Z boson mass. The abs() function might be useful. Also, we need to know
            # the position of this suitable Z Boson in the List of the Z_candidates.
            # Therefore, we are looking for an index to access the specific Z Boson by 
            # Z_candidates["E"][index]. np.argmin() function might be your option to choose.
            z_boson_masses = np.array(Z_candidates["mass"])
            nearest_to_z_mass_index = 0

            # writing the found quantities into the dataframe
            df.loc[i, ("Z1", "E")] = Z_candidates["E"][nearest_to_z_mass_index]
            df.loc[i, ("Z1", "px")] = Z_candidates["px"][nearest_to_z_mass_index]
            df.loc[i, ("Z1", "py")] = Z_candidates["py"][nearest_to_z_mass_index]
            df.loc[i, ("Z1", "pz")] = Z_candidates["pz"][nearest_to_z_mass_index]


            # explicitly naming the label of used leptons for the reconstruction of
            # the Z1 Boson
            Z1_label = Z_candidates["label"][nearest_to_z_mass_index]

            # How many possible combinations are left when you consider the point, that
            # both reconstructed Z bosons don't share any leptons... which should be the usual case?
            for j, label in enumerate(Z_candidates["label"]):
                # Why is it important to skip the Steps that are fulfilling this
                # requirement?
                if label[0] in Z1_label or label[1] in Z1_label:
                    continue

                df.loc[i, ("Z2", "E")] = Z_candidates["E"][j]
                df.loc[i, ("Z2", "px")] = Z_candidates["px"][j]
                df.loc[i, ("Z2", "py")] = Z_candidates["py"][j]
                df.loc[i, ("Z2", "pz")] = Z_candidates["pz"][j]


        # the mixed channel is considered in the following lines
        if event.event_information.channel == 2:
            # the difference to the four muon or electron channel is that for our
            # calculation of the Z Bosons we have to consider the flavour of the
            # provided leptons

            # accessing the Energy, px, ... like above: muon_1.E, muon_1.px, ...
            muon_1, muon_2 = get_leptons_by_flavour(event, 0)
            electron_1, electron_2 = get_leptons_by_flavour(event, 1)

            # calculate the Energy, px, py, pz and the mass of a Z boson that decayed
            # into two muons
            E_muons = 0.0   # Calculate the total px considering both muons
            Px_muons = 0.0  # Calculate the total py considering both muons
            Py_muons = 0.0  # Calculate the total pz considering both muons
            Pz_muons = 0.0  # Calculate the total E considering both muons
            M_muons = calculate_invariant_mass(E_muons, Px_muons, Py_muons, Pz_muons)

            # calculate the Energy, px, py, pz and the mass of a Z boson that decayed
            # into two electrons
            E_electrons  = 0.0 # Calculate the total px considering both electrons
            Px_electrons = 0.0 # Calculate the total py considering both electrons
            Py_electrons = 0.0 # Calculate the total pz considering both electrons
            Pz_electrons = 0.0 # Calculate the total E considering both electrons
            M_electrons = calculate_invariant_mass(E_electrons,
                                                            Px_electrons,
                                                            Py_electrons,
                                                            Pz_electrons)


            from_muons, from_electrons = "Z1", "Z2"

            # formulate the condition (similar to the case above, but less complicated)
            # when choosing the Z1 candidate, that have to be the one nearest to the
            # nominal Z mass
            condition_to_reassign = False
            if condition_to_reassign:
                from_muons, from_electrons = "Z2", "Z1"
            
            

            # writing the found quantities into the dataframe
            df.loc[i, (from_muons, "E")] = E_muons
            df.loc[i, (from_muons, "px")] = Px_muons
            df.loc[i, (from_muons, "py")] = Py_muons
            df.loc[i, (from_muons, "pz")] = Pz_muons

            # writing the found quantities into the dataframe
            df.loc[i, (from_electrons, "E")] =  E_electrons
            df.loc[i, (from_electrons, "px")] = Px_electrons
            df.loc[i, (from_electrons, "py")] = Py_electrons
            df.loc[i, (from_electrons, "pz")] = Pz_electrons

        # Calculate the Energy, px, py, pz of the Higgs candidate given two reconstructed Z Bosons:
        # You can access those e.g. df.loc[i, ("Z1", "E")] event wise...
    # Actually, as we don't need to consider different Z candidates anymore, we can do it column wise
    # by replacing "i" with ":" and accessing them by df.loc[:, ("Z1", "E")], df.loc[:, ("Z2", "E")] and so on.

    df.loc[:, ("four_lep", "E")] = 0.0  # Calculate the total E considering both reconstructed Z Bosons
    df.loc[:, ("four_lep", "px")] = 0.0 # Calculate the total px considering both reconstructed Z Bosons
    df.loc[:, ("four_lep", "py")] = 0.0 # Calculate the total py considering both reconstructed Z Bosons
    df.loc[:, ("four_lep", "pz")] = 0.0 # Calculate the total pz considering both reconstructed Z Bosons

    # the mass of this Higgs candidate works the same way
    return df

In [ ]:
dfs = perform_reconstruction(dfs)
dfb = perform_reconstruction(dfb)
dfm = perform_reconstruction(dfm)



Da dieser Schritt länger dauert, wäre es ratsam nach einer erfolgreichen Rekonstruktion den nun vollständigen Datensatz zwischenzuspeichern und beim neu starten das Notebook diesen rekonstruierten Datensatz direkt zu laden und die Rekonstruktion überspringen. Folgender Code kann für das Speichern ausgeführt werden. Wichtig ist hier, dass der neue Ordner bereits vorhanden sein sollte, wenn er genutzt werden will.


In [ ]:
# dfs.to_csv("data_reconstructed/MC_201012_H_to_ZZ_to_4L_[100,151].csv.gz", index=False)  # signal MC
# dfb.to_csv("data_reconstructed/MC_2012_ZZ_to_4L_[100,151].csv.gz", index=False)  # signal MC
# dfm.to_csv("data_reconstructed/CMS_Run22[B,C]_[100,151].csv.gz", index=False)


Kommen wir nun zu der Visualisierung der Ergebnisse der durchgeführten Rekonstruktion. Die Visualisierung erfolgt in Form von Histogrammen, da diese sehr viele Informationen prägnant zusammenfassen können. Sollte aus dem Schulunterricht noch keine Berührungspunkte damit geben, so kann das notwendige Wissen auch in Form einer Einführung seitens des Betreuers erfolgen.



Für die tatsächliche Visualisierung der betrachteten Größen - zunächst einmal unabhängig von der durchgeführten Messung - nutzen wir die `plot_quantities` Hilfsfunktion, deren Anwendung in der Regel anhand des folgenden Beispielcodes an anderen Stellen mit den gleichen Argumenten weiterverwendet werden kann.




In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14, 'xtick.labelsize': 14, 'ytick.labelsize': 14})


from utils import plot_quantities


Die rekonstruierten physikalischen Objekte sind die beiden Z-Bosonen und der Higgs Kandidat `four_lep`. Interessant sind vor allem die Verteilungen der Massen.



In [ ]:
print("Signal")
plot_quantities(df=[dfs], 
                column=["Z1", "Z2", "four_lep"], quantity="mass", 
                label=["Signal"], unit="GeV", suptitle="Signal Simulation",
                yscale=["log", "log", "log"],
                hist_range=[(0, 140), (0, 80), (100, 150)])  # optional

print("Untergrund")
plot_quantities(df=[dfb], 
                column=["Z1", "Z2", "four_lep"], quantity="mass", 
                label=["Background"], unit="GeV", suptitle="Untergrund Simulation",
                yscale=["log", "log", "log"],
                hist_range=[(0, 140), (0, 80), (100, 150)])  # optional

print("Messung")
plot_quantities(df=[dfm], 
                column=["Z1", "Z2", "four_lep"], quantity="mass", 
                label=["Unprocessed Measurement"], unit="GeV", suptitle="Durchgeführte Messung",
                yscale=["log", "log", "log"],
                hist_range=[(0, 140), (0, 80), (100, 150)])  # optional


Welche Form der Verteilungen können beobachtet werden?

Gibt es Unterschiede zwischen den Untergrund und Signalsimulationen

Gibt es einen Unterschied zwischen den Masseverteilungen der Z-Bosonen untereinander und der vier Leptonen invarianten Masse?

Das Ziel ist nun diese Datensätze derart zu bereinigen, dass im besten Fall eine Resonanz, eine Überhöhung von einem möglichen Higgs Boson zu sehen sein wird. Dieses Bereinigen soll auf den simulierten Ereignissen durchgeführt werden (!), damit seitens des Experimentators - in diesem fall Dir - keine Vorurteile in die Analyse eingebracht werden. In der Regel werden die Daten auch nicht wie in dem obigen Fall gezeigt angeschaut - aus didaktischen Gründen ist es aber hilfreich, um die Motivation hinter den durchgeführten Filterschritten zu zeigen.

Die Motivation zu einzelnen Filtern, vor allem der relativen Isolation, sollte interaktiv mit dem Betreuer motiviert werden. Einen sehr kurzen Abriss über die Motivation erfolgt ebenfalls in Textform.

Die Anwendung einzelner Filter erfolgt aneinandergereiht über 
  ```
  (dataframe
      .pipe(filtername, filterargumente)
      .pipe(filername, filterargumente)
      ...)
  ```

Folgende Filter stehen dir zu Verfügung:

  * `LeptonFilter.min_pt_lepton` mit `min_pt_electron` und `min_pt_muon` als notwendige Argumente
  * `LeptonFilter.relative_isolation_lepton`, mit dem notwendigen `relative_isolation_value` Argument
  * `EventFilter.min_lepton_number` ist ein argumentloser Filter ebenso wie
  * `EventFilter.neutral_charge` die nach jedem Einsatz von anderen Filtern angewendet werden müssen, da diese alle Ereignisse entfernen, in denen aufgrund der Anwendung von anderen Filtern nicht mehr vier Leptonen vorhanden sind oder keine elektrisch neutrale Ladungskombination mehr existiert.
  * `EventFilter.z_masses` mit den eingrenzenden Massenwerten `z1_mass_min`, `z1_mass_max`, `z2_mass_min`, `z2_mass_max`

##### Minimaler Transversalimpuls:

Dieser Filter entfernt alle Leptonen, die einen Transversalimpuls (in x-y Richtung kleiner als einen bestimmten Wert besitzen. Teilchen mit geringem transversalen Impuls sind im besten Fall oft nur schlecht rekonstruierte Leptonen. Ebenso können es Teilchen sein, die fälschlicherweise als Leptonen identifiziert wurden. Durch das Entfernen dieser Leptonen wird sichergestellt, dass die Leptonen, die für die Rekonstruktion verwendet werden, die bestmöglichen sind.


Zur Visualisierung eignet es sich immer einen vorher-nachher Vergleich anzuschauen. Hierzu wieder die Hilfsfunktion zum Darstellen von den Verteilungen.


In [ ]:
plot_quantities(df=[dfs,
                   (dfs.pipe(LeptonFilter.min_pt_lepton, min_pt_electron=0, min_pt_muon=0)
                        .pipe(EventFilter.min_lepton_number)
                        .pipe(EventFilter.neutral_charge))], 
                column=["Z1", "Z2", "four_lep"], quantity="mass", 
                label=["Signal before Cut", "Signal after Cut"], unit="GeV",
                yscale=["log", "log", "log"],
                hist_range=[(0, 140), (0, 80), (100, 150)])  # optiona

plot_quantities(df=[dfb,
                   (dfb.pipe(LeptonFilter.min_pt_lepton, min_pt_electron=0, min_pt_muon=0)
                        .pipe(EventFilter.min_lepton_number)
                        .pipe(EventFilter.neutral_charge))], 
                column=["Z1", "Z2", "four_lep"], quantity="mass", 
                label=["Background before Cut", "Background after Cut"], unit="GeV",
                yscale=["log", "log", "log"],
                hist_range=[(0, 140), (0, 80), (100, 150)])  # optiona

Das Ziel bei der Wahl der Werte sollte sein, eine größtmögliche Reduktion des Untergrundes bei bestmöglicher Beibehaltung des Signals zu erreichen. Ein Problem, das damit auftauchen kann, ist, dass die begrenze Größe des Messdatensatzes die Freiheit bei der Wahl der spezifischen Grenzen der Filter einschränkt. Die bestmöglichen Filterergebnisse bei den simulierten Datensätzen haben nur wenig Nutzen, wenn kaum noch Ereignisse in der tatsächlichen Messung vorhanden sind.

In [ ]:
# Place for visualize the impacts of individual cuts and experiments on specific filter values here

Nach der Fahl der Werte für die Filtergrenzen sollten diese nicht mehr geändert werden und in der gleichen Form auf den Datensatz mit der Messung angewendet werden. 

##### Relative Isolation und Z-Bosonen Massen:

Bei dem Filter nach der relativen Isolation wird darauf geschaut, wie gut die einzelnen Leptonen von allen anderen Teilchen in einem Ereignis isoliert sind. Sind diese gut isoliert (kleiner `relpfiso` Wert) so gibt es in ihrer näheren Umgebung keine weiteren Teilchen, die die Rekonstruktion in dem Detektor verfälschen könnten, besteht diese Isolation nicht, so kann es passieren, dass zum Beispiel statt Leptonen aus einem Z-Bosonen Zerfall Leptonen aus einem Jet verwendet werden und so die Datensätze kontaminieren.

Für die Massen der Z-Bosonen sollten die Werte derart gewählt werden, dass eines der Bosonen möglichst eine Masse um $91\, \mathrm{GeV}$ besitzt und das andere Z-Boson eine deutlich kleinere Masse besitzt. Dadurch können Ereignisse entfernt werden, bei denen der aus den beiden Z-Bosonen rekonstruierter Higgs Kandidat eine deutlich kleinere Masse als die erwartete Masse besitzt.

In [ ]:
plot_quantities(df=[dfs,
                   (dfs.pipe(LeptonFilter.min_pt_lepton, min_pt_electron=0, min_pt_muon=0)
                        .pipe(EventFilter.min_lepton_number)
                        .pipe(EventFilter.neutral_charge)
                        .pipe(EventFilter.z_masses, z1_mass_min=0, z1_mass_max=9999, z2_mass_min=0, z2_mass_max=9999)))], 
                column=["Z1", "Z2", "four_lep"], quantity="mass", 
                label=["Signal before Cut", "Signal after Cut"], unit="GeV",
                yscale=["log", "log", "log"],
                hist_range=[(0, 140), (0, 80), (100, 150)])  # optiona

plot_quantities(df=[dfb,
                   (dfb.pipe(LeptonFilter.min_pt_lepton, min_pt_electron=0, min_pt_muon=0)
                        .pipe(EventFilter.min_lepton_number)
                        .pipe(EventFilter.neutral_charge)
                        .pipe(EventFilter.z_masses, z1_mass_min=0, z1_mass_max=9999, z2_mass_min=0, z2_mass_max=9999)))], 
                column=["Z1", "Z2", "four_lep"], quantity="mass", 
                label=["Background before Cut", "Background after Cut"], unit="GeV",
                yscale=["log", "log", "log"],
                hist_range=[(0, 140), (0, 80), (100, 150)])  # optiona

In [ ]:
# Place for visualize the impacts of individual cuts and experiments on specific filter values here

Mit den gewählten Werten für die alle Filter können nun diese final auf die simulierten Datensätze und nun auch die eigentliche Messung durchgeführt werden.

In [ ]:
min_pt_electron, min_pt_muon = 9999, 9999
relative_isolation = 9999
z1_mass_min, z1_mass_max, z2_mass_min, z2_mass_max = 9999, 9999, 9999, 9999

def apply_all_filters(dataframe):
    return (dataframe.pipe(LeptonFilter.min_pt_lepton, min_pt_electron=min_pt_electron, min_pt_muon=min_pt_muon)
                     .pipe(LeptonFilter.relative_isolation_lepton, relative_isolation_value=relative_isolation)
                     .pipe(EventFilter.neutral_charge)
                     .pipe(EventFilter.min_lepton_number)
                     .pipe(EventFilter.z_masses, z1_mass_min=z1_mass_min, z1_mass_max=z1_mass_max, z2_mass_min=z2_mass_min, z2_mass_max=z2_mass_max))

reduced_dfs = apply_all_filters(dfs)
reduced_dfb = apply_all_filters(dfb)
reduced_dfm = apply_all_filters(dfm)

Damit kommen wir zum letzten Punkt in dem Notebook: Dem tatsächlichen Vergleich zwischen den simulierten erwarteten Ergebnissen und der tatsächlichen Messung. Die Idee hierzu ist, dass die simulierten Ergebnisse an die Luminosität (der Datenmenge der Messung) abhängig von dem jeweiligen Zerfallskanal skaliert werden, da diese oftmals viel mehr Ereignisse enthalten, um bessere Statistik zu haben (wie aus den obigen Verteilungen entnommen werden kann). Die Skalierung als solche kann mitunter ziemlich mühsam sein. Aus diesem Grund wird diese hier explizit nicht durchgeführt, sondern in einer Hilfsfunktion zusammengefasst, die es dann nur noch anzuwenden gilt:

In [ ]:
from utils import get_scaled_bins_mc_data_comparison

In [ ]:
def plot_mc_data_comparison(df_data, df_mc_sig, df_mc_bkg, obj="four_lep", quantity="mass", bins=15, hist_range=(106, 151), 
                            xlabel=r"$m_{4\ell}$ in GeV"):

    bins_sig, bins_bkg, bins_measurement, edges, measurement_x = get_scaled_bins_mc_data_comparison(df_data,  
                                                                                                    df_mc_sig,  
                                                                                                    df_mc_bkg,  
                                                                                                    obj,  
                                                                                                    quantity,  
                                                                                                    bins,  
                                                                                                    hist_range)
    # Plotting starts here
    fig, ax = plt.subplots(1, 1, figsize=(7, 5))

    ax.errorbar(measurement_x, bins_measurement, yerr=np.sqrt(bins_measurement), fmt="ko", label="Measurement")
    ax.fill_between(edges, bins_sig + bins_bkg, bins_bkg, step="post", color="none", 
                    label=r"Signal MC ($m_{\mathrm{H}}=125\, \mathrm{GeV}$)", lw=2,facecolor='orangered')
    ax.fill_between(edges, bins_bkg, step="post", color="royalblue", label="Background MC")

    ax.set(xlabel=xlabel, ylabel=f"Events/{round((edges[-1] - edges[0]) / len(edges[1:]), 1)} GeV", 
           xlim=(edges[0], edges[-1]), ylim=(0, None))
    ax.legend()
    plt.tight_layout()
    plt.show()

    return bins_bkg, bins_measurement

In [ ]:
bkg_sim_sim, measurment_bins = plot_mc_data_comparison(reduced_dfm, reduced_dfs, reduced_dfb, obj="four_lep", quantity="mass")

Die Aussage darüber, ob diese von Dir beobachtete Überhöhung tatsächlich signifikant ist und auf ein Higgs Boson mit einer Masse von $125\, \mathrm{GeV}$ schließen lässt, ist ein guter Diskussionsanfang mit dem Betreuer, der je nach Interesse und vorhandenem Vorwissens seitens der Schülerin oder des Schülers nach eigenem Ermessen weitergehen kann.

In der Teilchenphysik wird so eine Abweichung von dem erwarteten Untergrund in Einheit der Standardabweichungen angegeben. Die Motivation dahinter sollte interaktiv visualisiert werden.

Eine grobe Abschätzung der Signifikanz kann zum Beispiel durch $Z = \frac{s}{\sqrt{b}}$ bestimmt werden. Gäbe es in unserem Fall sehr viel Untergrund, dann könnte eine leicht abweichende Überhöhung davon nicht unbedingt einem Signal zugeordnet werden, da diese Überhöhung auch zufällig sein könnte. Die Signifikanz als die Überhöhung der beobachten Signalereignisse gegenüber der Unsicherheit auf den erwarteten Untergrund (daher $\sqrt{b}$ und nicht einfach $b$) ist in der Teilchenphysik eine oft benutzte Größe um festzustellen ob es einen Indiz ($3\sigma$) auf bestimmte (neue) Prozesse oder Teilchen gibt. Ab $5\sigma$ spricht man in der Teilchenphysik dann von einer Entdeckung. 

Schätze die Signifikanz für die von dir beobachtete Überhöhung ab. die Summe vom erwartetn Untergrund kann über `bkg_sim_sim.sum()` erhalten werden, ebenso wie die Summe der Ereignisse der beobachteten Messungen `measurment_bins.sum()`

In [ ]:
# Calculate the significance of the observed exess in the four lepton invariant mass spectrum